In [1]:
# Dependencies
from splinter import Browser
from bs4 import BeautifulSoup as bs
import time
import requests
import os
import pandas as pd
import pprint
import pymongo

In [2]:
# URL for news site
url = "https://mars.nasa.gov/news/"

In [3]:
# https://splinter.readthedocs.io/en/latest/drivers/chrome.html
!which chromedriver

/usr/local/bin/chromedriver


In [4]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [5]:
browser.visit(url)

In [6]:
html = browser.html

In [7]:
# Create BeautifulSoup object; parse with 'html.parser'
soup = bs(html, 'html.parser')

In [8]:
# Find the title of the latest article
title = soup.find('div', class_='content_title')
news_title = title.text
print(news_title)

Mars InSight Lander's 'Mole' Pauses Digging


In [9]:
# Find the paragraph text of the latest aricle
paragraph = soup.find('div', class_='article_teaser_body')
news_p = paragraph.text
print(news_p)

After the mission's heat probe began hammering last week, it appears to have hit a rock. The team is analyzing data before they hammer again.


In [11]:
# URL for space images site
url2 = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'

browser.visit(url2)

browser.click_link_by_id("full_image")

html2 = browser.html

# Create BeautifulSoup object; parse with 'html.parser'
soup2 = bs(html2, 'html.parser')

# Find the title of the latest article
image = soup2.find('img', class_='fancybox-image')['src']
print(image)

/spaceimages/images/mediumsize/PIA16227_ip.jpg


In [12]:
# Get url of featured image
featured_image_url = 'https://www.jpl.nasa.gov' + image
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA16227_ip.jpg


In [13]:
# URL for Mars Weather twitter
url3 = 'https://twitter.com/marswxreport?lang=en'

In [14]:
browser.visit(url3)
html3 = browser.html

In [15]:
# Create BeautifulSoup object; parse with 'html.parser'
soup3 = bs(html3, 'html.parser')

In [16]:
# Find the weather information on Mars based on the most recent tweet
weather = soup3.find('p', class_='TweetTextSize TweetTextSize--normal js-tweet-text tweet-text')
mars_weather = weather.text
print(mars_weather)

InSight sol 96 (2019-03-05) low -95.9ºC (-140.6ºF) high -19.1ºC (-2.4ºF)
pressure at 7.20 hPapic.twitter.com/SFpZrpPEZI


In [17]:
# URL for Mars Facts
url4 = 'https://space-facts.com/mars/'


In [18]:
browser.visit(url4)
html4 = browser.html


In [19]:
# Create BeautifulSoup object; parse with 'html.parser'
soup4 = bs(html4, 'html.parser')


In [20]:
# Get table info and convert to html
tables = pd.read_html(html4)
info_table = tables[0]
info_table = info_table.set_index(0)
info_table = info_table.to_html(index_names=False, header=False)
info_table

'<table border="1" class="dataframe">\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.42 x 10^23 kg (10.7% Earth)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.52 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-153 to 20 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>\n  </tbody>\n</table>'

In [21]:
# URL for USGS Astrogeology
url5 = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

In [22]:
browser.visit(url5)
html5 = browser.html

In [23]:
# Create BeautifulSoup object; parse with 'html.parser'
soup5 = bs(html5, 'html.parser')

In [24]:
def retrieve_hemis():
    # URL for USGS Astrogeology
    url5 = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
    
    # Visit URL and parse html
    browser.visit(url5)
    html5 = browser.html
    soup5 = bs(html5, 'html.parser')
    
    # find the articles
    articles = soup5.find_all('div', class_='description')[0:4]
    
    # create list object to store output
    imgs = []
    
    # iterate over articles
    for article in articles:
        img = {}
        href = article.h3.text
        browser.click_link_by_partial_text(href)
        html5 = browser.html
        soup5 = bs(html5, 'html.parser')
        img['title'] = href
        img['img_url'] = soup5.find('a', target='_blank')['href']
        imgs.append(img)
        
        #restart process
        browser.visit(url5)
    
    return(imgs)


In [25]:
imgs = retrieve_hemis()

In [26]:
print(imgs)

[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]
